In [172]:
import numpy as np

class MoveError(Exception):
    pass

class Game:
    BOARD_SIZE = 7
    TOTAL_GOLD_PIECES = 8
    TOTAL_SILVER_PIECES = 12
    
    
    def __init__(self, start): # start = starting player (1 or -1)
        
        self.board = np.zeros((self.BOARD_SIZE, self.BOARD_SIZE)); # initialize a 7x7 board populated with 0's
        
        self.board[2:5, 2:5] = 1 # populate the center 3x3 tiles with 1's (gold ships)
        self.board[3][3] = 2 # populate the center tile with 2 (flag)
        self.board[2:5, [0, 6]] = -1 # populate the middle 3 outer horizontal tiles with -1 (silver ships)
        self.board[[0, 6], 2:5] = -1 # populate the middle 3 outer vertical tiles with -1 (silver ships)
        
        self.goldPieces = [(i, j) for i in range(2, 5) for j in range(2, 5)] # contains all the positions of the gold pieces
        self.goldPieces.remove((3, 3)) # flag is a special piece
        self.silverPieces = [(i, j) for i in range(7) for j in range(7) if self.board[i, j] == -1] # contains all the positions of the silver pieces
        self.flag = (3,3) # flag is a special piece
        
        self.toMove = start # player to move first
        self.numMoves = 0 # number of moves already made by the player
        
        self.gameState = 0 # gamestate => 0 if game is in progress, 1 if gold wins, -1 if silver wins

        self.prevMove = None # this variable is used to store the first movement made in a turn, to ensure that a piece isn't moved twice
        
    # def hasPiecesBetween(self, old_x, old_y, new_x, new_y):
    #     # Check if positions are not parallel horizontally or vertically
    #     if old_x != new_x and old_y != new_y:
    #         raise ValueError("Positions must be parallel either horizontally or vertically")

    #     dx = 1 if new_x > old_x else -1 if new_x < old_x else 0 # Determine the direction of movement (horizontal left/right)
    #     dy = 1 if new_y > old_y else -1 if new_y < old_y else 0 # Determine the direction of movement (vertical up/down)

        
    #     current_x, current_y = old_x + dx, old_y + dy
    #     while (current_x, current_y) != (new_x, new_y): # Iterate through positions between (old_x, old_y) and (new_x, new_y)
    #         if self.board[current_x, current_y] != 0:
    #             return True  # Found a piece between
    #         current_x += dx 
    #         current_y += dy
    #     if self.board[current_x, current_y] != 0: # for the case of moving a single square
    #         return True 

    #     return False  # No obstacles found
    
    def move(self, originalPos, nextPos):
        old_x,old_y = originalPos;
        new_x, new_y = nextPos;
        
        if not (0 <= old_x <= 6 and 0 <= old_y <= 6 and 0 <= new_x <= 6 and 0 <= new_y <= 6) : # either the old pos or new pos is out of bounds for the 7x7 board
            raise MoveError("Out of Bounds")
        
        if self.board[old_x][old_y] == 0 : # if the selected tile to move doesn't have a piece on it
            raise MoveError("No Piece in that tile")
        
        if (self.board[old_x][old_y] * self.toMove < 0) : # piece selected isn't the current player's pieces
            #explanation: because gold's pieces are positive and silver's are negative, this condition will only be negative if the current player selects the oposing player's pieces
            raise MoveError("Invalid Piece to Move")
        
        euclidian_distance = (old_x - new_x)**2 + (old_y - new_y)**2 # euclidian distance squared
        
        if (euclidian_distance > 2)  or ((old_x,old_y) == (new_x,new_y)) :
            # if it's not in a tile immediate to the original position. Or moving in place
            raise MoveError("Invalid Position to move to")
        
        # finished testing general invalidations, now decide what move is being made
        
        if old_x == new_x or old_y == new_y :            # if (self.hasPiecesBetween(old_x, old_y, new_x, new_y)) : # if there are any pieces in between the old and new positions
            #     raise MoveError("Invalid Movement. There are other pieces impeding this movement")
            # if self.prevMove == (old_x,old_y) : # if there are any pieces in between the old and new positions
            #     raise MoveError("Invalid Movement. Cannot move piece that has already moved this turn")# horizontal move 
            # if (self.hasPiecesBetween(old_x, old_y, new_x, new_y)) : # if there are any pieces in between the old and new positions
            #     raise MoveError("Invalid Movement. There are other pieces impeding this movement")
            # if self.prevMove == (old_x,old_y) : # if there are any pieces in between the old and new positions
            #     raise MoveError("Invalid Movement. Cannot move piece that has already moved this turn")
            
            if self.toMove == 1 : # gold pieces
                if self.board[old_x][old_y] == 1 :
                    self.goldPieces.remove((old_x,old_y))
                    self.goldPieces.append((new_x,new_y))
                    self.prevMove = (new_x,new_y)
                elif (self.numMoves == 0) : # moving flag. since it's not a piece, if it passed the earlier checks, it must be the flag
                    self.flag = (new_x,new_y) # will only ever enter this condition if the player can select the flag piece, aka, the player owns the gold pieces
                    self.numMoves += 2
                    if new_x == 0 or new_x == 6 or new_y == 0 or new_y == 6 :
                        self.gameState = 1 #flag piece reached the border, gold won
                else :
                    raise MoveError("Invalid Movement. Not enough movement left to move flag")
                
            else : # silver pieces
                self.silverPieces.remove((old_x,old_y))
                self.silverPieces.append((new_x,new_y))
                self.prevMove = (new_x,new_y)
            
            
            self.board[new_x][new_y] = self.board[old_x][old_y] # if no errors were raised, change the position
            self.board[old_x][old_y] = 0
            
            self.numMoves += 2
            
            if self.numMoves >= 2 :
                self.numMoves = 0 # reset move counter
                self.prevMove = None
                self.toMove *= -1 # after moving twice or moving the flag, give the turn to the other player
            return True # successful move
                
        # diagonal capture
        
        if euclidian_distance == 2 : # only going to be equal to 2 at the immediate diagonals
            if self.board[new_x][new_y] == 0 :
                raise MoveError("Invalid Movement. can't move into a diagonal empty space")
            if self.board[new_x][new_y] * self.board[old_x][old_y] > 0 : # will only result in positive if the pieces are of the same side
                raise MoveError("Invalid Movement. can't capture your own pieces")
            if self.numMoves != 0 :
                raise MoveError("Invalid Movement. Not enough movement to capture")
            
            if self.toMove == 1 and self.board[new_x][new_y] == -1 : # gold pieces capturing a silver piece
                if self.board[old_x][old_y] == 1 :
                    self.goldPieces.remove((old_x,old_y))
                    self.goldPieces.append((new_x,new_y))
                else : # moving flag
                    self.flag = (new_x,new_y) # will only ever enter this condition if the player can select the flag piece, aka, the player owns the gold pieces
                
                self.silverPieces.remove((new_x,new_y)) # silver piece was captured
                
            elif self.toMove == -1 and self.board[new_x][new_y] == 1 :
                self.silverPieces.remove((old_x,old_y))
                self.silverPieces.append((new_x,new_y))
                
                self.goldPieces.remove((new_x,new_y)) # gold piece was captured
                
            elif self.toMove == -1 and self.board[new_x][new_y] == 2 :
                self.silverPieces.remove((old_x,old_y))
                self.silverPieces.append((new_x,new_y))
                
                self.flag = None;
                self.gameState = -1 #flag was captured, silver won
                
            self.board[new_x][new_y] = self.board[old_x][old_y] # if no errors were raised, change the position
            self.board[old_x][old_y] = 0
                
            self.toMove *= -1 # after capturing, give the turn to the other player
            
    def evaluate_board(self, player, missing_pieces, count_capturable, count_defended, flag_defense, flag_how_close_to_edge) : # evaluates the board and returns the heuristic for the current player
        # the variables are the weights of each heuristic, 0 if heuristic is disabled
        heuristic = 0
        missing_pieces_value = 0
        capturable_value = 0
        defended_value = 0
        flag_defense_value = 0
        flag_distance_to_edge_value = 0
        
        if player == 1 : # gold pieces
            if self.gameState == 1 :
                return np.inf # infinity if it's a winning game state
            elif self.gameState == -1 :
                return np.NINF
            
            if missing_pieces != 0 : # heuristic isn't disabled
                missing_pieces_value = self.count_pieces_missing(-1) * missing_pieces # count the number of missing silver pieces
            if count_capturable != 0 : # heuristic isn't disabled
                capturable_value =  self.count_capturable(1) * count_capturable # count the number of pieces gold can capture
            if count_defended != 0 : # heuristic isn't disabled
                defended_value = self.count_defended(1) * count_defended # count the number of gold pieces that are defended
            if flag_defense != 0 : # heuristic isn't disabled
                flag_defense_value = self.flag_defense() * flag_defense # how many gold pieces are defending the flag
            if flag_how_close_to_edge != 0 : # heuristic isn't disabled
                flag_how_close_to_edge_value = -self.flag_how_close_to_edge() * flag_how_close_to_edge # how many edges are visible to the flag
            
        else : # silver pieces
            if self.gameState == -1 :
                return np.inf # infinity if it's a winning game state
            elif self.gameState == 1 :
                return np.NINF
            
            if missing_pieces != 0 : # heuristic isn't disabled
                missing_pieces_value = self.count_pieces_missing(1) * missing_pieces # count the number of missing silver pieces
            if count_capturable != 0 : # heuristic isn't disabled
                capturable_value =  self.count_capturable(-1) * count_capturable # count the number of pieces silver can capture
            if count_defended != 0 : # heuristic isn't disabled
                defended_value = self.count_defended(-1) * count_defended # count the number of silver pieces that are defended
            if flag_defense != 0 : # heuristic isn't disabled
                flag_defense_value = -self.flag_defense() * flag_defense # how many gold pieces are defending the flag
            if flag_how_close_to_edge != 0 : # heuristic isn't disabled
                flag_how_close_to_edge_value = -self.flag_how_close_to_edge() * flag_how_close_to_edge # flag distance from edge
                

        print(f"{player} - Pieces Missing: {missing_pieces_value}")
        print(f"{player} - Capturable Pieces: {capturable_value}")
        print(f"{player} - Defended Pieces: {defended_value}")
        print(f"{player} - Flag Defense: {flag_defense_value}")
        print(f"{player} - Flag How Close to Edge: {flag_how_close_to_edge_value}")
        
        heuristic = missing_pieces_value + capturable_value + defended_value + flag_defense_value + flag_distance_to_edge_value
        
        return heuristic
            
    def count_pieces_missing(self,player) : # how many pieces have been captured from a player
        if player == 1 : # gold pieces
            return self.TOTAL_GOLD_PIECES - len(self.goldPieces) 
        else :
            return self.TOTAL_SILVER_PIECES - len(self.silverPieces)

    def count_capturable(self, player) :
        result = 0

        if player == 1:  # gold pieces
            for gold_piece in self.goldPieces: # for every gold piece
                x, y = gold_piece
                # Check if there is a capturable opponent piece in the diagonally adjacent positions
                for dx in [-1, 1]:
                    for dy in [-1, 1]:
                        new_x, new_y = x + dx, y + dy
                        if 0 <= new_x < self.BOARD_SIZE and 0 <= new_y < self.BOARD_SIZE: # assures that it doesn't go out of bounds
                            if self.board[new_x][new_y] == -1:
                                result += 1

        else:  # silver pieces
            for silver_piece in self.silverPieces: # for every silver piece
                x, y = silver_piece
                # Check if there is a capturable opponent piece in the diagonally adjacent positions
                for dx in [-1, 1] :
                    for dy in [-1, 1] :
                        new_x, new_y = x + dx, y + dy
                        # Note: This condition might not be necessary since if the flag gets to one of the edges, the game is over and gold wins
                        if 0 <= new_x < self.BOARD_SIZE and 0 <= new_y < self.BOARD_SIZE : # assures that it doesn't go out of bounds
                            if self.board[new_x][new_y] == 1 :
                                result += 1
                            elif self.board[new_x][new_y] == 2 : # can capture the flag
                                result += 10
                                
        return result 
    
    def count_defended(self, player) : # pieces of the same team that are "covering" eachother, aka, diagonally adjacent
        result = 0
        pieces = None;
        directions = [(-1, -1), (1, 1), (-1, 1), (1, -1)] # diagonal

        if player == 1 :  # gold pieces
            pieces = self.goldPieces.copy()
            pieces.append(self.flag)

        else :  # silver pieces
            pieces = self.silverPieces

        for piece in pieces :  # for every piecee
            x, y = piece
            # Check if there is a piece defending in the diagonally adjacent positions
            for dx, dy in directions: 
                new_x, new_y = x + dx, y + dy
                if 0 <= new_x < self.BOARD_SIZE and 0 <= new_y < self.BOARD_SIZE :  # assures that it doesn't go out of bounds
                    if self.board[new_x][new_y] * self.board[x][y] > 0 : # same side
                        result += 1
                        break # piece is defended, don't count it twice

        return result
    
    def flag_defense(self) : # counts how many gold pieces are defending the flag
        flag_x, flag_y = self.flag
        defense_score = 0
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (1, 1), (-1, 1), (1, -1)] # ortogonal and diagonal

        for dx, dy in directions:
            new_x, new_y = flag_x + dx, flag_y + dy
            euclidean_distance = (flag_x - new_x) ** 2 + (flag_y - new_y) ** 2
            if 0 <= new_x < self.BOARD_SIZE and 0 <= new_y < self.BOARD_SIZE: # ensure that it's within board boundaries
                if self.board[new_x][new_y] == 1 : # if there is a gold piece
                    defense_score += euclidean_distance  # adds the euclidian distance
                elif self.board[new_x][new_y] == -1 : # if there is a silver piece
                    defense_score -= euclidean_distance * 2  # subtracts the euclidian distance times 2

        return defense_score
    
    def flag_how_close_to_edge(self):
        flag_x, flag_y = self.flag

        up_distance = flag_y
        down_distance = 6 - flag_y
        left_distance = flag_x
        right_distance = 6 - flag_x

        return 3 - max(up_distance, down_distance, left_distance, right_distance)


    
    def possible_moves_for_piece(self, piece):
        x,y = piece
        possible_moves = []

        # Check orthogonally adjacent positions
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < self.BOARD_SIZE and 0 <= new_y < self.BOARD_SIZE and self.board[new_x][new_y] == 0:
                possible_moves.append((new_x, new_y))

        # Check diagonally adjacent positions with an enemy piece
        for dx, dy in [(-1, -1), (1, 1), (-1, 1), (1, -1)]:
            new_x, new_y = x + dx, y + dy
            print(f"Checking ({new_x}, {new_y})")
            if 0 <= new_x < self.BOARD_SIZE and 0 <= new_y < self.BOARD_SIZE and self.board[new_x][new_y] * self.board[x][y] < 0 and self.board[new_x][new_y] != 0:  
                possible_moves.append((new_x, new_y))

        return possible_moves
    
    def minimax(self, depth, maximizing_player, alpha, beta):
        if depth == 0 or self.gameState != 0:
            return self.evaluate_board(maximizing_player, 1, 1, 1, 1, 1), None

        legal_moves = self.generate_all_moves(maximizing_player)

        if maximizing_player == 1:
            best_value = np.NINF
            best_move = None
            for move in legal_moves:
                new_game_state = self.copy_and_apply_move(move)
                value, _ = new_game_state.minimax(depth - 1, -1, alpha, beta)
                if value > best_value:
                    best_value = value
                    best_move = move
                alpha = max(alpha, best_value)
                if beta <= alpha:
                    break
            return best_value, best_move  # Return both the heuristic value and the move

        else:
            best_value = np.inf
            best_move = None
            for move in legal_moves:
                new_game_state = self.copy_and_apply_move(move)
                value, _ = new_game_state.minimax(depth - 1, 1, alpha, beta)
                if value < best_value:
                    best_value = value
                    best_move = move
                beta = min(beta, best_value)
                if beta <= alpha:
                    break
            return best_value, best_move  # Return both the heuristic value and the move


    def generate_all_moves(self, player):
        possible_moves = []
        pieces = self.goldPieces if player == 1 else self.silverPieces
        
        for piece in pieces:
            for move in self.possible_moves_for_piece(piece):
                new_x, new_y = move
                if 0 <= new_x < self.BOARD_SIZE and 0 <= new_y < self.BOARD_SIZE:
                    print(f"Checking move {move} for piece {piece}")
                    possible_moves.append((piece, move))
                
        return possible_moves

    def copy_and_apply_move(self, move):
        new_game_state = Game(self.toMove)
        new_game_state.board = np.copy(self.board)
        new_game_state.goldPieces = self.goldPieces.copy()
        new_game_state.silverPieces = self.silverPieces.copy()
        new_game_state.flag = self.flag
        new_game_state.toMove = self.toMove
        new_game_state.numMoves = self.numMoves
        new_game_state.gameState = self.gameState
        new_game_state.prevMove = self.prevMove

        originalPos, nextPos = move
        new_game_state.move(originalPos, nextPos)

        return new_game_state

In [173]:
import tkinter as tk
from tkinter import messagebox


class BattleScreen(tk.Frame):
    def __init__(self, root, controls, startPlayer):
        tk.Frame.__init__(self, root)
        self.controls = controls
        self.game = Game(startPlayer)
        self.buttons = [[None] * 7 for _ in range(7)]
        self.selected_piece = None
        self.controls.update_turn_label(startPlayer)
        
        for row in range(7):
            for col in range(7):
                button = tk.Button(self, width=5, height=2, command=lambda r=row, c=col: self.on_button_click(r, c))
                self.configure_button_color(button, row, col)    
                button.grid(row=row, column=col)
                self.buttons[row][col] = button
                
    def configure_button_color(self, button, row, col):
        if self.game.board[row][col] == 1:
            button.config(bg="yellow")
        elif self.game.board[row][col] == -1:
            button.config(bg="blue")
        elif self.game.board[row][col] == 2:
            button.config(bg="orange")
        else:
            button.config(bg="grey")
    
    def update_board(self):
        for row in range(7):
            for col in range(7):
                self.configure_button_color(self.buttons[row][col], row, col)

    def on_button_click(self, row, col):
        if self.selected_piece is None:
            self.selected_piece = (row, col)
            self.buttons[row][col].config(relief=tk.SOLID, bd=3, highlightbackground="red") # Highlight the selected button with a red border
            return
        else:
            try:
                original_pos = self.selected_piece
                next_pos = (row, col)
                self.game.move(original_pos, next_pos)
                
                for piece in self.game.goldPieces :
                    print(f"gold piece {piece} can move to {self.game.possible_moves_for_piece(piece)}")
                self.update_board()
                self.controls.update_turn_label(self.game.toMove)  # update turn label in Controls

                if self.game.toMove == -1:  # Check if it's AI's turn after player's move
                    self.handle_ai_move()
                    
            except MoveError as e:
                print(f"Movement Error: {e}")

            
            self.buttons[self.selected_piece[0]][self.selected_piece[1]].config(relief=tk.RAISED, bd=1, highlightbackground="grey") # remove the red border from the previously selected button
            self.selected_piece = None # dereference selected piece
            
            print(self.game.evaluate_board(player = 1, missing_pieces = 1, count_capturable = 1, count_defended = 1, flag_defense = 1, flag_how_close_to_edge = 1))

            
            if self.game.gameState == 1 :
                self.show_winner_popup("Gold")
            elif self.game.gameState == -1:
                self.show_winner_popup("Silver")
    
    def handle_ai_move(self):
        best_value, best_move = self.game.minimax(depth=3, maximizing_player=-1, alpha=np.NINF, beta=np.inf)

        # Print information for debugging
        print("Best Move:", best_move)
        print("Best Value:", best_value)
        print("Current Player:", self.game.toMove)
        print("Game State:", self.game.gameState)
        
        original_pos = best_move[0]
        next_pos = best_move[1]

        try:
            self.game.move(original_pos, next_pos)
        except MoveError as e:
            print(f"MoveError: {e}")
            print("Game State after invalid move:", self.game.gameState)

    def show_winner_popup(self, winner):
        message = f"{winner.capitalize()} Wins!"
        messagebox.showinfo("Game Over", message)

        
        for row in range(7):
            for col in range(7):
                self.buttons[row][col].config(state=tk.DISABLED) # disable the game board

        self.master.destroy() # close the game

In [174]:
class Controls(tk.Frame):
    def __init__(self, root):
        tk.Frame.__init__(self, root)

        self.quit_button = tk.Button(self, text="Quit", width=6, command=root.destroy)
        self.quit_button.pack()

        self.turn_label = tk.Label(self, text="Turn: Gold")
        self.turn_label.pack()

    def update_turn_label(self, player):
        turn_text = "Turn: Gold" if player == 1 else "Turn: Silver"
        self.turn_label.config(text=turn_text)


In [175]:
class GameApp:
    def __init__(self, root, startPlayer):
        self.root = root
        self.root.title("Breakthru")

        self.controls = Controls(self.root)
        
        self.battle_screen = BattleScreen(self.root, self.controls, startPlayer)
        
        self.battle_screen.pack()
        self.controls.pack()

In [176]:
def main():
    root = tk.Tk()
    app = GameApp(root, 1)
    app.battle_screen.handle_ai_move()  # Make the AI make the first move
    root.mainloop()

In [177]:
main()

Checking (1, 1)
Checking (3, 3)
Checking (1, 3)
Checking (3, 1)
gold piece (2, 2) can move to [(1, 2), (2, 1)]
Checking (1, 2)
Checking (3, 4)
Checking (1, 4)
Checking (3, 2)
gold piece (2, 3) can move to [(1, 3)]
Checking (1, 3)
Checking (3, 5)
Checking (1, 5)
Checking (3, 3)
gold piece (2, 4) can move to [(1, 4), (2, 5)]
Checking (2, 1)
Checking (4, 3)
Checking (2, 3)
Checking (4, 1)
gold piece (3, 2) can move to [(3, 1)]
Checking (2, 3)
Checking (4, 5)
Checking (2, 5)
Checking (4, 3)
gold piece (3, 4) can move to [(4, 4), (3, 5)]
Checking (3, 1)
Checking (5, 3)
Checking (3, 3)
Checking (5, 1)
gold piece (4, 2) can move to [(5, 2), (4, 1)]
Checking (3, 2)
Checking (5, 4)
Checking (3, 4)
Checking (5, 2)
gold piece (4, 3) can move to [(5, 3), (4, 4)]
Checking (3, 4)
Checking (5, 6)
Checking (3, 6)
Checking (5, 4)
gold piece (4, 5) can move to [(3, 5), (5, 5), (4, 4), (3, 6)]
1 - Pieces Missing: 0
1 - Capturable Pieces: 1
1 - Defended Pieces: 9
1 - Flag Defense: 10
1 - Flag How Close to